# Vanilla Option Pricing Practice
* 이 파일에서는 Black Scholes Model 을 이용하여 옵션 가격을 구하는 방법을 구현
* 콜옵션을 기준으로 작성
* 내재 변동성 산출 방법 까지 구현

In [13]:
# 사용 하는 파이썬 내부 모듈 선언
import math
import datetime

In [3]:
"""
함수 설명
 * 표준 정규 분포의 확률 계산 -> 엑셀에는 관련 함수가 있으므로 그것을 그대로 사용 하면됨
"""
def get_norm_prob(x):
    a1 = 0.254829592
    a2 = -0.284496736
    a3 = 1.4214143741
    a4 = -1.453152027
    a5 = 1.061405429
    p = 0.3275911

    sign = 1
    if x < 0: sign = -1

    x = math.fabs(x) / math.sqrt(2.0)

    t = 1.0 / (1.0 + p * x)
    y = 1.0 - (((((a5 * t + a4) * t ) + a3) * t + a2) * t + a1) * t * math.exp(- x * x)
    return 0.5*(1.0 + sign * y)

In [10]:
"""
함수 설명
 * Black-Scholes model 구현

입력 변수
 * stock_price    : 기초 자산의 현재가격
 * strike_price   : 행사가격
 * interest_rate  : 무위험 이자율(잔존기간을 고려하여 금리 커프에서 가져옴)
 * vol            : 기초자산의 변동성
 * dividend       : 기초자산의 배당률
 * terminate_time : 옵션의 잔존기간(만기일 - 계산일의 연환산으로 여기서는 1년을 365로 사용)
 * option_type    : 옵션의 타입(1 : 콜옵션, 2 : 풋옵션)
"""
def get_option_price(stock_price, strike_price, interest_rate, vol, dividend, terminate_time, option_type):
    d1 = ( math.log(stock_price / strike_price) + (interest_rate - dividend + (vol * vol) / 2) * terminate_time ) / (vol * math.sqrt(terminate_time))
    d2 = d1 - vol * math.sqrt(terminate_time)
        
    if option_type == 1:  # Call
        return math.exp(-dividend * terminate_time) * stock_price * get_norm_prob(d1) - math.exp(- interest_rate * terminate_time) * strike_price * get_norm_prob(d2)
    else: # Put
        return - math.exp(-dividend * terminate_time) * stock_price * get_norm_prob(-d1) + math.exp( - interest_rate * terminate_time) * strike_price * get_norm_prob(-d2)

In [12]:
"""
함수 설명
 * 잔존만기(terminate_time)에 따른 이자율을 구하는 방법으로 잔존만기가 정해진 tenor값과 동일하면 그에 해당하는 rate 값을 반환해준다.
 * 만약 정해진 두 tenor 사이에 있는 경우 보간법을 이용하여 rate를 산출 한다.(terminate_time 이 0.45인 경우 0.25와 0.5 사이에 있으므로 보간법을 이용하여 산출)

입력 변수
 * discount_curve_tenor : 할인 금리 커브의 만기별 tenor 만 만들어 놓은 리스트
 * discount_curve_rate  : 할인 금리 커브의 이자율만 모아놓은 리스트로 위의 tenor 리스트의 같은 인덱스 끼리 매칭(예 : tenor가 0.25 일때 인덱스는 1이고 그 인덱스와 동일한 rate 는 0.009189)
"""
def get_ir_interpolation(discount_curve_tenor, discount_curve_rate, terminate_time):
    if(len(discount_curve_tenor) == 1):
        return discount_curve_rate[0]
    if(discount_curve_tenor[-1] <= terminate_time):
        return discount_curve_rate[:-1]

    for i, tenor in enumerate(discount_curve_tenor):
        if tenor == terminate_time:
            return discount_curve_rate[i]
        else:
            if tenor > terminate_time:
                return discount_curve_rate[i-1] + (terminate_time - discount_curve_tenor[i-1]) / (discount_curve_tenor[i] - discount_curve_tenor[i-1]) * (discount_curve_rate[i] - discount_curve_rate[i-1])

In [6]:
"""
함수 설명
 * 잔존만기를 연 환산 해주는 함수
 * exercise_date - valuation_date 를 연환산을 하기 위해 365로 나눠줌
 * 두 입력 변수가 문자열로 선언되어 있으므로 그것을 날짜로 만들어 주는 파이썬 내부 모듈 사용 -> 엑셀에서는 관련 함수가 있으므로 확인 후 사용

입력 변수
 * valuation_date : 할인 금리 커브의 만기별 tenor 만 만들어 놓은 리스트
 * exercise_date  : 할인 금리 커브의 이자율만 모아놓은 리스트로 위의 tenor 리스트의 같은 인덱스 끼리 매칭(예 : tenor가 0.25 일때 인덱스는 1이고 그 인덱스와 동일한 rate 는 0.009189)
"""
def get_terminate_time(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y%m%d")
    end = datetime.datetime.strptime(end_date, "%Y%m%d")
    return ((end - start).days) / 365

In [15]:
# 발행정보
valuation_date = '20210906'
stock_price = 306000
strike_price = 300000
historical_vol = 0.28323
implied_vol_from_kiwoom = 0.32977432030953
dividend = 0.033
exercise_date = '20211014'
option_type = 1  # call = 1, put = 2

mkt_option_price = 15600.0       # 20210906 당시 시장에서 실재 거래된 옵션의 가격

# 할인금리 커브
discount_curve_tenor = [0.00278, 0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75,5,5.25,5.5,5.75,6,6.25,6.5,6.75,7,7.25,7.5,7.75,8,8.25,8.5,8.75,9,9.25,9.5,9.75,10,12,15,20
]
discount_curve_rate = [0.0076,0.009189,0.01086,0.011712,0.012462,0.012963,0.013464,0.01384,0.014217,0.014418,0.014619,0.01482,0.015022,0.015167,0.015313,0.015457,0.015603,0.015718,0.015831,0.015946,0.016061,0.016115,0.016141,0.016178,0.016223,0.016264,0.016305,0.016346,0.016387,0.016426,0.016464,0.016502,0.01654,0.016579,0.016618,0.016657,0.016695,0.016732,0.01677,0.016809,0.016851,0.017113,0.016598,0.01611537
]

# 잔존 만기 및 그에 해당하는 이자율 산출
terminate_time = get_terminate_time(valuation_date, exercise_date)
interest_rate = get_ir_interpolation(discount_curve_tenor, discount_curve_rate, terminate_time)

In [19]:
# 역사적 변동성을 이용한 옵션 가격 산출
option_price = get_option_price(stock_price, strike_price, interest_rate, historical_vol, dividend, terminate_time, option_type)
print(f'Option Price = {option_price}')

Option Price = 13818.234900245268


In [20]:
# 옵션의 Vega 산출
def get_option_vega(vol, stock_price, strike_price, interest_rate, dividend, terminate_time):
    d1 = (math.log(stock_price/strike_price) + (interest_rate - dividend + (vol * vol) / 2)*terminate_time) / (vol * math.sqrt(terminate_time))
    return - math.exp(dividend * terminate_time) * stock_price * (1/math.sqrt(2* math.pi)) * math.exp(- d1 * d1 / 2) * math.sqrt(terminate_time)

In [21]:
# 내재변동성 산출 시 시행착오방법을 이용하는데 중간에 사용하는 함수(로직 이해 필요 X)
def compare_mktprice_pricefbs(mkt_option_price, option_price_bs):
    return mkt_option_price - option_price_bs

In [23]:
# implied vol 산출 로직

error_term = 0.00001     # 오차 범위
n_trial = 1000           # 시행 횟수
x0 = historical_vol

dx = 0
x1 = 0

implied_vol = 0.0

# 시행착오 로직
for i in range(n_trial):
    dx = get_option_vega(x0, stock_price, strike_price, interest_rate, dividend, terminate_time)
    tmp_option_price = get_option_price(stock_price, strike_price, interest_rate, x0, dividend, terminate_time, 1)
    x1 = x0 - (compare_mktprice_pricefbs(mkt_option_price, tmp_option_price) / dx)

    if x1 < 0:
        x1 = 0.0001
    elif x1 > 2:
        x1 = 2
    
    if math.fabs(x1 - x0) < error_term:
        if x1 == x0 and x1 == 2:
            implied_vol = 0.0001
        else:
            implied_vol = x1
        break
    x0 = x1

print(f"산출 된 내재 변동성 : {implied_vol}")
print(f'산출 된 내재 변동성으로 산출 한 옵션 가격 : {get_option_price(stock_price, strike_price, interest_rate, implied_vol, dividend, terminate_time, 1 )}')
print("시장에서 관찰 된 옵션 가격과 내재변동성으로 산출 된 가격이 거의 동일하게 산출 된것을 확인 할 수 있음")

산출 된 내재 변동성 : 0.329774153403158
산출 된 내재 변동성으로 산출 한 옵션 가격 : 15599.999620566203
시장에서 관찰 된 옵션 가격과 내재변동성으로 산출 된 가격이 거의 동일하게 산출 된것을 확인 할 수 있음
